In [ ]:
from faker import Faker
import random
import mysql.connector
from datetime import datetime, timedelta

In [ ]:
db_config = {
    "host": "localhost",
    "user": "john",
    "password": "blueharb0r!",
    "database": "blueharbor"
}

In [ ]:
fake = Faker()
random.seed(42)

In [ ]:
def connect():
    return mysql.connector.connect(**db_config)

In [ ]:
def seed_tax_rates(cursor):
    countries = [
        ("United Kingdom", 0.20, 0.05),
        ("France", 0.20, 0.07),
        ("Germany", 0.19, 0.06),
        ("Netherlands", 0.21, 0.05),
        ("United States", 0.00, 0.08),
        ("China", 0.13, 0.10),
        ("Japan", 0.10, 0.04),
        ("India", 0.18, 0.12)
    ]

    cursor.executemany("""
        INSERT INTO tax_rates (country, vat_rate, customs_duty_rate)
        VALUES (%s, %s, %s)
        ON DUPLICATE KEY UPDATE vat_rate=VALUES(vat_rate),
                                customs_duty_rate=VALUES(customs_duty_rate);
    """, countries)
    print(f"Inserted/updated {cursor.rowcount} tax rates.")


In [ ]:
def seed_products(cursor):
    product_list = [
        ("Electronics - Smartphone", "Electronics"),
        ("Electronics - Laptop", "Electronics"),
        ("Clothing - Jacket", "Apparel"),
        ("Clothing - Shoes", "Apparel"),
        ("Furniture - Chair", "Home"),
        ("Furniture - Table", "Home"),
        ("Toys - Board Game", "Toys"),
        ("Food - Chocolate", "Groceries")
    ]

    cursor.executemany("""
        INSERT INTO products (product_name, product_category)
        VALUES (%s, %s);
    """, product_list)
    print(f"Inserted {cursor.rowcount} products.")


In [ ]:
def seed_shipments_and_invoices(cursor, n_shipments=500):
    # products and countries in DB
    cursor.execute("SELECT product_id FROM products;")
    product_ids = [row[0] for row in cursor.fetchall()]

    cursor.execute("SELECT country, vat_rate, customs_duty_rate FROM tax_rates;")
    country_data = cursor.fetchall()
    countries = [row[0] for row in country_data]
    tax_lookup = {row[0]: {"vat": row[1], "duty": row[2]} for row in country_data}

    incoterms = ["CIF", "FOB", "DDP", "EXW"]
    
    shipments, invoices = [], []

    for _ in range(n_shipments):
        origin, destination = random.sample(countries, 2)
        product_id = random.choice(product_ids)
        declared_value = round(random.uniform(500, 10000), 2)
        weight_kg = round(random.uniform(10, 1000), 2)
        ship_date = fake.date_between(start_date="-2y", end_date="today")
        incoterm = random.choice(incoterms)

        shipments.append((origin, destination, product_id, declared_value, weight_kg, ship_date, incoterm))

    cursor.executemany("""
        INSERT INTO shipments (origin_country, destination_country, product_id, declared_value, weight_kg, shipment_date, incoterm)
        VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, shipments)

    cursor.execute("SELECT shipment_id, destination_country, declared_value FROM shipments;")
    shipment_rows = cursor.fetchall()

    for shipment_id, dest, value in shipment_rows:
        rates = tax_lookup.get(dest, {"vat": 0.0, "duty": 0.0})
        vat = round(value * rates["vat"], 2)
        duty = round(value * rates["duty"], 2)
        invoice_value = round(value + vat + duty, 2)
        invoice_date = fake.date_between(start_date="-2y", end_date="today")
        tax_flag = "OK" if random.random() > 0.05 else "MISSING_VAT"

        invoices.append((shipment_id, invoice_date, invoice_value, vat, duty, tax_flag))

    cursor.executemany("""
        INSERT INTO invoices (shipment_id, invoice_date, invoice_value, vat_amount, customs_amount, tax_flag)
        VALUES (%s, %s, %s, %s, %s, %s);
    """, invoices)

    print(f"Inserted {len(shipments)} shipments and {len(invoices)} invoices.")

    


In [ ]:
if __name__ == "__main__":
    conn = connect()
    cursor = conn.cursor()

    seed_tax_rates(cursor)
    seed_products(cursor)
    seed_shipments_and_invoices(cursor, n_shipments=300)

    conn.commit()
    cursor.close()
    conn.close()
    print("Data seeded into MySQL.")